In [314]:
import radarsimpy

print("`RadarSimPy` used in this example is version: " + str(radarsimpy.__version__))

`RadarSimPy` used in this example is version: 12.4.0


In [315]:
import numpy as np
from radarsimpy import Radar, Transmitter, Receiver

In [316]:
az_angle = np.arange(-20, 21, 1)
az_pattern = 20 * np.log10(np.cos(az_angle / 180 * np.pi) ** 500) + 20

el_angle = np.arange(-20, 21, 1)
el_pattern = 20 * np.log10((np.cos(el_angle / 180 * np.pi)) ** 400) + 20

_Plot the antenna patterns_


In [317]:
import plotly.graph_objs as go
from IPython.display import Image

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=az_angle,
        y=az_pattern,
        name="Azimuth",
    )
)
fig.add_trace(
    go.Scatter(
        x=el_angle,
        y=el_pattern,
        name="Elevation",
    )
)

fig.update_layout(
    title="Antenna Pattern",
    yaxis=dict(title="Amplitude (dB)", range=[-90, 22]),
    xaxis=dict(title="Angle (deg)", range=[-90, 90]),
)

# uncomment this to display interactive plot
fig.show()

# display static image to reduce size on radarsimx.com
# img_bytes = fig.to_image(format="jpg", scale=2)
# display(Image(img_bytes))

In [318]:
max_range = 5000 # Maximum unambiguous range
range_res = 50 # Required range resolution
pulse_bw = 3e8/(2*range_res) # Pulse bandwidth
pulse_width = 1/pulse_bw # Pulse width
prf = 3e8/(2*max_range) # Pulse repetition frequency
prp = 1/prf
fs = 2*pulse_bw # Sampling rate
fc=10e9
num_pulse=10

total_samples = int(prp*num_pulse*fs)
sample_per_pulse = int(total_samples/num_pulse)

mod_t = np.arange(0, total_samples)/fs
amp = np.zeros_like(mod_t)

amp[mod_t<=pulse_width]=1

tx_channel = dict(
    location=(0, 0, 0),
    azimuth_angle=az_angle,
    azimuth_pattern=az_pattern,
    elevation_angle=el_angle,
    elevation_pattern=el_pattern,
    amp=amp,
    mod_t=mod_t,
)

tx = Transmitter(
    f=fc,
    t=1/prf,
    tx_power=67, # dBm
    pulses=num_pulse,
    channels=[tx_channel],
)

In [319]:
rx_channel = dict(
    location=(0, 0, 0),
    azimuth_angle=az_angle,
    azimuth_pattern=az_pattern,
    elevation_angle=el_angle,
    elevation_pattern=el_pattern,
)

In [320]:
rx = Receiver(
    fs=fs,
    noise_figure=12,
    rf_gain=20,
    load_resistor=500,
    baseband_gain=30,
    channels=[rx_channel],
)

In [321]:
radar = Radar(transmitter=tx, receiver=rx)

### Targets

The propertities of targets are defined here. There are 3 targets in this simulation. The locations of the targets are defined through $(x, y, z)$ coordinates in meters, and the speeds of the targets are defined trough $(v_x, v_y, v_z)$ in $m/s$. The propertites of the targets also includes radar cross-section (RCS (dBsm)) and phase (degree).


In [322]:
target_1 = dict(location=(2000, 0, 0), speed=(0, 0, 0), rcs=10, phase=0)
target_2 = dict(location=(3000, 0, 0), speed=(0, 0, 0), rcs=10, phase=0)


targets = [target_1, target_2]

## Simulate Baseband Signals

Use the `simulator.simc` module to simulate the baseband samples from the defined radar system and targets.

The output baseband data is a dict including the timestamp and baseband. Both of them are 3-D matrix:

`[channels, pulses, ADC samples]`


In [323]:
from radarsimpy.simulator import sim_radar

data = sim_radar(radar, targets)
timestamp = data["timestamp"]
baseband = data["baseband"]+data["noise"]

_Plot the baseband samples_


In [324]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=timestamp[0, 0, :],
        y=np.real(baseband[0, 0, :]),
        name="I",
    )
)
fig.add_trace(
    go.Scatter(
        x=timestamp[0, 0, :],
        y=np.imag(baseband[0, 0, :]),
        name="Q",
    )
)

fig.update_layout(
    title="I/Q Baseband Signals",
    yaxis=dict(title="Amplitude (V)"),
    xaxis=dict(title="Time (s)"),
)

# uncomment this to display interactive plot
fig.show()

# display static image to reduce size on radarsimx.com
# img_bytes = fig.to_image(format="jpg", scale=2)
# display(Image(img_bytes))

## Radar Signal Processing




In [325]:
from scipy import signal

In [326]:
matchingcoeff = amp[amp!=0]

In [327]:
range_profile = np.zeros_like(baseband, dtype=np.complex128)

for p_idx in range(0, num_pulse):
    range_profile[0, p_idx, :] = signal.convolve(baseband[0, p_idx, :], matchingcoeff, mode="same")

In [328]:
range_profile_integrated = np.sum(np.abs(range_profile[0,:,:]), axis=0)

In [330]:
range_axis = mod_t*3e8/2

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=range_axis,
        y=20*np.log10(range_profile_integrated),
        name="I",
    )
)

fig.update_layout(
    title="I/Q Baseband Signals",
    yaxis=dict(title="Amplitude (dB)"),
    xaxis=dict(title="Range (m)"),
)

# uncomment this to display interactive plot
fig.show()